Hi, every one. I am almost new to the kaggle, and I do the ML and DL jobs during my work. I find the kaggle as best for me to make it done. With no more words , I used the catboost and Tensorflow to make the Titanic prediction, and I got the top 7% for it, and I want to share with others what I have done and if is there any better suggestion for it , I want to discuss with you! Here we go!

In [4]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.3 MB/s eta 0:00:00


In [5]:
#import
import numpy as np
import pandas as pd
import hyperopt
from catboost import Pool, CatBoostClassifier, cv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [12]:
!ls sample_data

anscombe.json		      mnist_test.csv	     Titanic_test.csv
california_housing_test.csv   mnist_train_small.csv  Titanic_train.csv
california_housing_train.csv  README.md


In [103]:
#get the train and test data
train_df = pd.read_csv('sample_data/Titanic_train.csv')
test_df = pd.read_csv('sample_data/Titanic_test.csv')

In [52]:
train2_df = pd.read_csv('sample_data/Titanic_data.csv', sep=';')

In [53]:
train2_df

,Survived,Passenger Class,Name,Sex,Age,No of Siblings or Spouses on Board,No of Parents or Children on Board,Ticket Number,Passenger Fare,Cabin,Port of Embarkation,Life Boat
0,Yes,First,"Allen, Miss. Elisabeth Walton",Female,29,0,0,24160,"211,3375",B5,Southampton,2
1,Yes,First,"Allison, Master. Hudson Trevor",Male,"0,9167",1,2,113781,"151,55",C22 C26,Southampton,11
2,No,First,"Allison, Miss. Helen Loraine",Female,2,1,2,113781,"151,55",C22 C26,Southampton,NaN
3,No,First,"Allison, Mr. Hudson Joshua Creighton",Male,30,1,2,113781,"151,55",C22 C26,Southampton,NaN
4,No,First,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",Female,25,1,2,113781,"151,55",C22 C26,Southampton,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,No,Third,"Zabour, Miss. Hileni",Female,"14,5",1,0,2665,"14,4542",NaN,Cherbourg,NaN
1305,No,Third,"Zabour, Miss. Thamine",Female,NaN,1,0,2665,"14,4542",NaN,Cherbourg,NaN
1306,No,Third,"Zakarian, Mr. Mapriededer",Male,"26,5",0,0,2656,"7,225",NaN,Cherbourg,NaN
1307,No,Third,"Zakarian, Mr. Ortin",Male,27,0,0,2670,"7,225",NaN,Cherbourg,NaN


In [44]:
#get the train and test data
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

FileNotFoundError: ignored

In [104]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [105]:
#show the train data
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [106]:
#show how many the null value for each column
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [107]:
#for the train data ,the age ,fare and embarked has null value,so just make it -999 for it
#and the catboost will distinguish it
train_df.fillna(-999,inplace=True)
test_df.fillna(-999,inplace=True)

In [108]:
#now we will get the train data and label
x = train_df.drop('Survived',axis=1)
y = train_df.Survived


In [109]:
#show what the dtype of x, note that the catboost will just make the string object to categorical
#object inside
x.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [110]:
#choose the features we want to train, just forget the float data
cate_features_index = np.where(x.dtypes != float)[0]

In [111]:
#make the x for train and test (also called validation data)
xtrain,xtest,ytrain,ytest = train_test_split(x,y,train_size=.85,random_state=1234)

In [112]:
#let us make the catboost model, use_best_model params will make the model prevent overfitting
model = CatBoostClassifier(eval_metric='Accuracy',use_best_model=True,random_seed=42)

In [113]:
#now just to make the model to fit the data
model.fit(xtrain,ytrain,cat_features=cate_features_index,eval_set=(xtest,ytest))

Learning rate set to 0.029583
0:	learn: 0.8124174	test: 0.8059701	best: 0.8059701 (0)	total: 9.21ms	remaining: 9.2s
1:	learn: 0.8110964	test: 0.8059701	best: 0.8059701 (0)	total: 12.8ms	remaining: 6.39s
2:	learn: 0.8163804	test: 0.7985075	best: 0.8059701 (0)	total: 20.2ms	remaining: 6.71s
3:	learn: 0.8150594	test: 0.7985075	best: 0.8059701 (0)	total: 28.3ms	remaining: 7.04s
4:	learn: 0.8190225	test: 0.7985075	best: 0.8059701 (0)	total: 34.5ms	remaining: 6.87s
5:	learn: 0.8269485	test: 0.8059701	best: 0.8059701 (0)	total: 42.3ms	remaining: 7.01s
6:	learn: 0.8269485	test: 0.8059701	best: 0.8059701 (0)	total: 53.4ms	remaining: 7.57s
7:	learn: 0.8216645	test: 0.8059701	best: 0.8059701 (0)	total: 61.1ms	remaining: 7.58s
8:	learn: 0.8216645	test: 0.8059701	best: 0.8059701 (0)	total: 64.3ms	remaining: 7.08s
9:	learn: 0.8256275	test: 0.7985075	best: 0.8059701 (0)	total: 71.5ms	remaining: 7.08s
10:	learn: 0.8216645	test: 0.8134328	best: 0.8134328 (10)	total: 78.4ms	remaining: 7.05s
11:	learn: 0

In [114]:
#for the data is not so big, we need use the cross-validation(cv) for the model, to find how
#good the model is ,I just use the 10-fold cv
cv_data = cv(model.get_params(),Pool(x,y,cat_features=cate_features_index),fold_count=10)

CatBoostError: ignored

In [115]:
#show the acc for the model
print('the best cv accuracy is :{}'.format(np.max(cv_data["b'Accuracy'_test_avg"])))

NameError: ignored

In [116]:
#show the model test acc, but you have to note that the acc is not the cv acc,
#so recommend to use the cv acc to evaluate your model!
print('the test accuracy is :{:.6f}'.format(accuracy_score(ytest,model.predict(xtest))))

the test accuracy is :0.858209


In [117]:
#last let us make the submission,note that you have to make the pred to be int!
pred = model.predict(test_df)
pred = pred.astype(np.int)
submission = pd.DataFrame({'PassengerId':test_df['PassengerId'],'Survived':pred})

<ipython-input-117-fdcbae167326>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pred = pred.astype(np.int)


In [80]:
model.save_model('tatianic.cbm',
           format="cbm",
           export_parameters=None,
           pool=None)

In [121]:
pred = model.predict(test_df)
pred = pd.DataFrame(pred)
pred = pred.rename(columns={0: 'Survived'})
res = test_df.join(pred)


In [122]:
res

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,-999,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,-999,S,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,-999,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,-999,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,-999,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,-999.0,0,0,A.5. 3236,8.0500,-999,S,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,1
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,-999,S,0
416,1308,3,"Ware, Mr. Frederick",male,-999.0,0,0,359309,8.0500,-999,S,0


In [125]:
res.to_csv('titanic_res.csv')

In [79]:
train2_df[train2_df['Name'] == 'Peter, Master. Michael J']

,Survived,Passenger Class,Name,Sex,Age,No of Siblings or Spouses on Board,No of Parents or Children on Board,Ticket Number,Passenger Fare,Cabin,Port of Embarkation,Life Boat
1121,Yes,Third,"Peter, Master. Michael J",Male,NaN,1,1,2668,"22,3583",NaN,Cherbourg,C


In [69]:
train2_df

,Survived,Passenger Class,Name,Sex,Age,No of Siblings or Spouses on Board,No of Parents or Children on Board,Ticket Number,Passenger Fare,Cabin,Port of Embarkation,Life Boat
0,Yes,First,"Allen, Miss. Elisabeth Walton",Female,29,0,0,24160,"211,3375",B5,Southampton,2
1,Yes,First,"Allison, Master. Hudson Trevor",Male,"0,9167",1,2,113781,"151,55",C22 C26,Southampton,11
2,No,First,"Allison, Miss. Helen Loraine",Female,2,1,2,113781,"151,55",C22 C26,Southampton,NaN
3,No,First,"Allison, Mr. Hudson Joshua Creighton",Male,30,1,2,113781,"151,55",C22 C26,Southampton,NaN
4,No,First,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",Female,25,1,2,113781,"151,55",C22 C26,Southampton,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,No,Third,"Zabour, Miss. Hileni",Female,"14,5",1,0,2665,"14,4542",NaN,Cherbourg,NaN
1305,No,Third,"Zabour, Miss. Thamine",Female,NaN,1,0,2665,"14,4542",NaN,Cherbourg,NaN
1306,No,Third,"Zakarian, Mr. Mapriededer",Male,"26,5",0,0,2656,"7,225",NaN,Cherbourg,NaN
1307,No,Third,"Zakarian, Mr. Ortin",Male,27,0,0,2670,"7,225",NaN,Cherbourg,NaN


In [30]:
#make the file to yourself's directory
submission.to_csv('catboost.csv',index=False)

Above all the catboost have done! catboost is based on boosting method and it is really amazing for the imputer data!
At last, If there is any problem in the code, you can tell me, I want to discuss all the possible to better solve the problem.  I also use the feature engineering for my work and use the Tensorflow to build a MLP to solve the problem, maybe because of my pool engineering feature, it may be not a better score(80.5%). I also use the ML method to solve it, and i find it is useful to standard the data to gaussian distribute(mean 0,std =1) but  min_max to 0-1 may not be better.